In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from finta import TA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import sklearn.ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
import pydotplus
from IPython.display import Image

### Stock Data Import and Cleaning (AMZN, META, TSLA)

In [2]:
# Read the csv file 
amzn_df = pd.read_csv(Path("Resources/amzn.csv"))

# Convert 'time' column from timestamp (seconds since epoch) to acutal time
amzn_df ['time'] = pd.to_datetime(
    amzn_df['time'],
    unit = 's',
    infer_datetime_format=True,
    utc=True
)    
# Convert timezone from UTC to Eastern Time
amzn_df['time'] = amzn_df['time'].dt.tz_convert('US/Eastern')

# Set 'time' column as the index 
amzn_df.set_index('time', inplace=True)

# Review df
amzn_df.head()          

,open,high,low,close,VWAP,Upper Band #1,Lower Band #1,Upper Band #2,Lower Band #2,Upper Band #3,...,Volume,Volume MA,EMA,Smoothing Line,Developing Poc,Developing VA High,Developing VA Low,Developing Poc.1,Developing VA High.1,Developing VA Low.1
time,,,,,,,,,,,,,,,,,,,,,
2012-03-14 09:30:00-04:00,9.1825,9.215995,9.057000,9.112999,9.128665,9.128665,9.128665,9.128665,9.128665,9.128665,...,73994680,107206355.0,9.597910,9.606610,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-15 09:30:00-04:00,9.1010,9.221500,9.015000,9.221500,9.152667,9.152667,9.152667,9.152667,9.152667,9.152667,...,83212340,103564913.0,9.594164,9.602464,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-16 09:30:00-04:00,9.1640,9.284000,9.117501,9.252500,9.218000,9.218000,9.218000,9.218000,9.218000,9.218000,...,98696480,95934268.0,9.590765,9.598431,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-19 09:30:00-04:00,9.1725,9.334000,9.150000,9.276000,9.253333,9.253333,9.253333,9.253333,9.253333,9.253333,...,78080380,92419724.0,9.587633,9.594651,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-20 09:30:00-04:00,9.2440,9.720500,9.144000,9.616500,9.493667,9.493667,9.493667,9.493667,9.493667,9.493667,...,183346180,94828072.0,9.587920,9.591678,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Filter column and only keep ones needed
amzn_df = amzn_df[['open','high','low','close','VWAP','Volume','Volume MA','EMA']]

#Rename EMA to 200EMA
amzn_df = amzn_df.rename(columns={'EMA':'200EMA'})

#Dropping NAN Values 
amzn_df = amzn_df.dropna()

# Review the DataFrame
amzn_df.head()

,open,high,low,close,VWAP,Volume,Volume MA,200EMA
time,,,,,,,,
2012-03-14 09:30:00-04:00,9.1825,9.215995,9.057000,9.112999,9.128665,73994680,107206355.0,9.597910
2012-03-15 09:30:00-04:00,9.1010,9.221500,9.015000,9.221500,9.152667,83212340,103564913.0,9.594164
2012-03-16 09:30:00-04:00,9.1640,9.284000,9.117501,9.252500,9.218000,98696480,95934268.0,9.590765
2012-03-19 09:30:00-04:00,9.1725,9.334000,9.150000,9.276000,9.253333,78080380,92419724.0,9.587633
2012-03-20 09:30:00-04:00,9.2440,9.720500,9.144000,9.616500,9.493667,183346180,94828072.0,9.587920


In [4]:
# Read the csv file 
meta_df = pd.read_csv(Path("Resources/meta.csv"))

# Convert 'time' column from timestamp (seconds since epoch) to acutal time
meta_df ['time'] = pd.to_datetime(
    meta_df['time'],
    unit = 's',
    infer_datetime_format=True,
    utc=True
)    
# Convert timezone from UTC to Eastern Time
meta_df['time'] = meta_df['time'].dt.tz_convert('US/Eastern')

# Set 'time' column as the index 
meta_df.set_index('time', inplace=True)

# Review df
meta_df.head()

,open,high,low,close,VWAP,Upper Band #1,Lower Band #1,Upper Band #2,Lower Band #2,Upper Band #3,...,Volume,Volume MA,EMA,Smoothing Line,Developing Poc,Developing VA High,Developing VA Low,Developing Poc.1,Developing VA High.1,Developing VA Low.1
time,,,,,,,,,,,,,,,,,,,,,
2012-05-18 09:30:00-04:00,42.05,45.00000,38.00,38.2318,40.410600,40.410600,40.410600,40.410600,40.410600,40.410600,...,580587776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-21 09:30:00-04:00,36.53,36.66000,33.00,34.0300,34.563333,34.563333,34.563333,34.563333,34.563333,34.563333,...,168309808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-22 09:30:00-04:00,32.61,33.59000,30.94,31.0000,31.843333,31.843333,31.843333,31.843333,31.843333,31.843333,...,102053808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-23 09:30:00-04:00,31.37,32.50000,31.36,32.0000,31.953333,31.953333,31.953333,31.953333,31.953333,31.953333,...,73721120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-24 09:30:00-04:00,32.95,33.21001,31.77,33.0300,32.670003,32.670003,32.670003,32.670003,32.670003,32.670003,...,50275872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Filter column and only keep ones needed
meta_df = meta_df[['open','high','low','close','VWAP','Volume','Volume MA','EMA']]

#Rename EMA to 200EMA
meta_df = meta_df.rename(columns={'EMA':'200EMA'})

#Dropping NAN Values 
meta_df = meta_df.dropna()

# Review the DataFrame
meta_df.head()

,open,high,low,close,VWAP,Volume,Volume MA,200EMA
time,,,,,,,,
2013-03-07 09:30:00-05:00,27.57001,28.67500,27.47000,28.57800,28.241000,74540128,46413816.0,25.669904
2013-03-08 09:30:00-05:00,28.42500,28.47000,27.73000,27.96001,28.053337,44198832,46896756.8,25.692691
2013-03-11 09:30:00-04:00,28.00999,28.64000,27.83000,28.14000,28.203333,35642064,46793423.2,25.717042
2013-03-12 09:30:00-04:00,28.10001,28.32001,27.60001,27.83000,27.916673,27569584,46303815.2,25.738067
2013-03-13 09:30:00-04:00,27.62000,27.64999,26.92000,27.08000,27.216663,39619440,43610005.6,25.751419


In [6]:
# Read the csv file 
tsla_df = pd.read_csv(Path("./Resources/tsla.csv"))

# Convert 'time' column from timestamp (seconds since epoch) to acutal time
tsla_df ['time'] = pd.to_datetime(
    tsla_df['time'],
    unit = 's',
    infer_datetime_format=True,
    utc=True
)    
# Convert timezone from UTC to Eastern Time
tsla_df['time'] = tsla_df['time'].dt.tz_convert('US/Eastern')

# Set 'time' column as the index 
tsla_df.set_index('time', inplace=True)
                      
# Review df
tsla_df.head()   

,open,high,low,close,VWAP,Upper Band #1,Lower Band #1,Upper Band #2,Lower Band #2,Upper Band #3,...,Volume,Volume MA,EMA,Smoothing Line,Developing Poc,Developing VA High,Developing VA Low,Developing Poc.1,Developing VA High.1,Developing VA Low.1
time,,,,,,,,,,,,,,,,,,,,,
2012-03-12 09:30:00-04:00,2.312664,2.419330,2.306665,2.400664,2.375553,2.375553,2.375553,2.375553,2.375553,2.375553,...,29467829,16848705.00,1.929784,1.921918,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-13 09:30:00-04:00,2.433997,2.439331,2.366664,2.405998,2.403998,2.403998,2.403998,2.403998,2.403998,2.403998,...,15024660,16731543.65,1.934522,1.925750,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-14 09:30:00-04:00,2.399998,2.399998,2.319998,2.352664,2.357553,2.357553,2.357553,2.357553,2.357553,2.357553,...,12771972,16011617.15,1.938683,1.929832,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-15 09:30:00-04:00,2.351998,2.365331,2.318664,2.333331,2.339109,2.339109,2.339109,2.339109,2.339109,2.339109,...,8573948,14366955.50,1.942610,1.934130,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-16 09:30:00-04:00,2.326664,2.392664,2.321998,2.354665,2.356442,2.356442,2.356442,2.356442,2.356442,2.356442,...,10938550,13249154.35,1.946710,1.938462,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Filter column and only keep ones needed
tsla_df = tsla_df[['open','high','low','close','VWAP','Volume','Volume MA','EMA']]

#Rename EMA to 200EMA
tsla_df = tsla_df.rename(columns={'EMA':'200EMA'})

#Dropping NAN Values 
tsla_df = tsla_df.dropna()

# Review the DataFrame
tsla_df.head()

,open,high,low,close,VWAP,Volume,Volume MA,200EMA
time,,,,,,,,
2012-03-12 09:30:00-04:00,2.312664,2.419330,2.306665,2.400664,2.375553,29467829,16848705.00,1.929784
2012-03-13 09:30:00-04:00,2.433997,2.439331,2.366664,2.405998,2.403998,15024660,16731543.65,1.934522
2012-03-14 09:30:00-04:00,2.399998,2.399998,2.319998,2.352664,2.357553,12771972,16011617.15,1.938683
2012-03-15 09:30:00-04:00,2.351998,2.365331,2.318664,2.333331,2.339109,8573948,14366955.50,1.942610
2012-03-16 09:30:00-04:00,2.326664,2.392664,2.321998,2.354665,2.356442,10938550,13249154.35,1.946710


### Importing Additional Data for ML Features Set (VIX, Search Trends)

In [8]:
# Read the csv file 
vix_df = pd.read_csv(Path("Resources/vix.csv"))

# Convert 'time' column from timestamp (seconds since epoch) to acutal time
vix_df ['time'] = pd.to_datetime(
    vix_df['time'],
    unit = 's',
    infer_datetime_format=True,
    utc=True
)    
# Convert timezone from UTC to Eastern Time
vix_df['time'] = vix_df['time'].dt.tz_convert('US/Eastern')

# Set 'time' column as the index 
vix_df.set_index('time', inplace=True)

# Calculate VIX EMA 
vix_df['VIX_40_EMA'] = TA.EMA(vix_df,40)

#Remove unnecessary columns
vix_df = vix_df[['close','VIX_40_EMA']]

#Rename close column
vix_df = vix_df.rename(columns={'close':'VIX_close'})

# Review df
vix_df.head()    

,VIX_close,VIX_40_EMA
time,,
2009-10-13 09:30:00-04:00,22.98999,22.989990
2009-10-14 09:30:00-04:00,22.85999,22.923365
2009-10-15 09:30:00-04:00,21.71999,22.502021
2009-10-16 09:30:00-04:00,21.42999,22.213586
2009-10-19 09:30:00-04:00,21.48999,22.054043


In [9]:
# Add VIX data to stock dfs
AMZN = pd.concat([amzn_df,vix_df],axis=1)
META = pd.concat([meta_df,vix_df],axis=1)
TSLA = pd.concat([tsla_df,vix_df],axis=1)


In [10]:
AMZN.head(3)

,open,high,low,close,VWAP,Volume,Volume MA,200EMA,VIX_close,VIX_40_EMA
time,,,,,,,,,,
2009-10-13 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.98999,22.989990
2009-10-14 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.85999,22.923365
2009-10-15 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.71999,22.502021


In [11]:
amzn_trend = pd.read_csv(Path('Search_Trends/Amazon.csv'))
amzn_trend = amzn_trend.set_index('Month')
amzn_trend = amzn_trend.rename(columns={'Amazon':'AMZN_search_trends'})
#display(amzn_trend.head())

meta_trend = pd.read_csv(Path('Search_Trends/Meta.csv'))
meta_trend = meta_trend.set_index('Month')
meta_trend = meta_trend.rename(columns={'Meta':'META_search_trends'})
#display(meta_trend.head())


tsla_trend = pd.read_csv(Path('Search_Trends/Tesla.csv'))
tsla_trend = tsla_trend.set_index('Month')
tsla_trend = tsla_trend.rename(columns={'tesla':'TSLA_search_trends'})
#display(tsla_trend.head())


In [12]:
# Create Month and Year cols based on index 
AMZN['Month'] = AMZN.index.month
AMZN['Year'] = AMZN.index.year

# Create Month and Year cols based on index (after formatting index as datetime)
amzn_trend.index = pd.to_datetime(amzn_trend.index)
amzn_trend['Month'] = amzn_trend.index.month
amzn_trend['Year'] = amzn_trend.index.year

#Create placeholder column for search trend data
AMZN['AMZN_search_trends'] = 0

# Create a dictionary from df2 where col1 and col2 are the keys, and value_df2 is the value
mapping_dict = amzn_trend.set_index(['Month', 'Year'])['AMZN_search_trends'].to_dict()

# Map the values from df2 to df1 based on col1 and col2
AMZN['AMZN_search_trends'] = AMZN.apply(lambda x: mapping_dict.get((x['Month'], x['Year']), x['AMZN_search_trends']), axis=1)

# Drop month & year columns
AMZN = AMZN.drop(columns=['Month','Year'])

# The df1 will now have values from df2 mapped based on matching col1 and col2.
AMZN

,open,high,low,close,VWAP,Volume,Volume MA,200EMA,VIX_close,VIX_40_EMA,AMZN_search_trends
time,,,,,,,,,,,
2009-10-13 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.98999,22.989990,0.0
2009-10-14 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.85999,22.923365,0.0
2009-10-15 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.71999,22.502021,0.0
2009-10-16 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.42999,22.213586,0.0
2009-10-19 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.48999,22.054043,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2023-07-18 09:30:00-04:00,132.71,133.86,131.355,132.83,132.681667,54969133.0,52859640.65,113.656816,13.30000,14.809797,0.0
2023-07-19 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.76000,14.758588,0.0
2023-07-20 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.99000,14.721096,0.0


In [13]:
# Create Month and Year cols based on index 
META['Month'] = META.index.month
META['Year'] = META.index.year

# Create Month and Year cols based on index (after formatting index as datetime)
meta_trend.index = pd.to_datetime(meta_trend.index)
meta_trend['Month'] = meta_trend.index.month
meta_trend['Year'] = meta_trend.index.year

#Create placeholder column for search trend data
META['META_search_trends'] = 0

# Create a dictionary from df2 where col1 and col2 are the keys, and value_df2 is the value
mapping_dict = meta_trend.set_index(['Month', 'Year'])['META_search_trends'].to_dict()

# Map the values from df2 to df1 based on col1 and col2
META['META_search_trends'] = META.apply(lambda x: mapping_dict.get((x['Month'], x['Year']), x['META_search_trends']), axis=1)

# Drop month & year columns
META = META.drop(columns=['Month','Year'])

# The df1 will now have values from df2 mapped based on matching col1 and col2.
META

,open,high,low,close,VWAP,Volume,Volume MA,200EMA,VIX_close,VIX_40_EMA,META_search_trends
time,,,,,,,,,,,
2009-10-13 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.98999,22.989990,0.0
2009-10-14 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.85999,22.923365,0.0
2009-10-15 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.71999,22.502021,0.0
2009-10-16 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.42999,22.213586,0.0
2009-10-19 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.48999,22.054043,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2023-07-18 09:30:00-04:00,310.875,314.2,307.6188,312.05,311.2896,20757323.0,26430892.9,216.492856,13.30000,14.809797,0.0
2023-07-19 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.76000,14.758588,0.0
2023-07-20 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.99000,14.721096,0.0


In [14]:
# Create Month and Year cols based on index 
TSLA['Month'] = TSLA.index.month
TSLA['Year'] = TSLA.index.year

# Create Month and Year cols based on index (after formatting index as datetime)
tsla_trend.index = pd.to_datetime(tsla_trend.index)
tsla_trend['Month'] = tsla_trend.index.month
tsla_trend['Year'] = tsla_trend.index.year

#Create placeholder column for search trend data
TSLA['TSLA_search_trends'] = 0

# Create a dictionary from df2 where col1 and col2 are the keys, and value_df2 is the value
mapping_dict = tsla_trend.set_index(['Month', 'Year'])['TSLA_search_trends'].to_dict()

# Map the values from df2 to df1 based on col1 and col2
TSLA['TSLA_search_trends'] = TSLA.apply(lambda x: mapping_dict.get((x['Month'], x['Year']), x['TSLA_search_trends']), axis=1)

# Drop month & year columns
TSLA = TSLA.drop(columns=['Month','Year'])

# The df1 will now have values from df2 mapped based on matching col1 and col2.
TSLA

,open,high,low,close,VWAP,Volume,Volume MA,200EMA,VIX_close,VIX_40_EMA,TSLA_search_trends
time,,,,,,,,,,,
2009-10-13 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.98999,22.989990,0.0
2009-10-14 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.85999,22.923365,0.0
2009-10-15 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.71999,22.502021,0.0
2009-10-16 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.42999,22.213586,0.0
2009-10-19 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.48999,22.054043,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2023-07-18 09:30:00-04:00,290.15,295.26,286.01,293.34,291.536667,112434713.0,1.370000e+08,214.699154,13.30000,14.809797,0.0
2023-07-19 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.76000,14.758588,0.0
2023-07-20 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.99000,14.721096,0.0


In [15]:
# Filter each dataframe to 10yr period
AMZN = AMZN.loc['05-2012':'05-2022']
META = META.loc['05-2012':'05-2022']
TSLA = TSLA.loc['05-2012':'05-2022']

#drop nan
AMZN = AMZN.dropna()
META = META.dropna()
TSLA = TSLA.dropna()


### Trading Algo (Signal Generation)

In [16]:
#set some options for display and troubleshooting
pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

In [17]:
# Define periods for fast and slow EMAs (triggers)
fast_ema = 9
slow_ema = 40

# Initialize list of ticker dfs for use in for loop
ticker_df = [AMZN,META,TSLA]

# For each ticker dataframe in the list:
for df in ticker_df:
    
    # Calculate % returns for later use
    df['pct_returns'] = df['close'].pct_change()
    
    # Calculate Fast EMA 
    df['Fast_EMA'] = TA.EMA(df,fast_ema)
    
    # Calculate Slow EMA
    df['Slow_EMA'] = TA.EMA(df,slow_ema)

    # Initialize empty Signal column with 0.0 default value
    df['Signal'] = 0.0

    # Generate Signal value ("long if fast_ema > slow_ema", otherwise flip short)
    df['Signal'] = np.where(
        df['Slow_EMA'] < df['Fast_EMA'], 1.0, -1.0)
    
#This loop should not generate Entry/Exit - this will not be needed until after ML predictions    
    # Create Entry/Exit column and fill with the differences (trades) from Signal column
    #df['Entry/Exit'] = df['Signal'].diff()
    
    #Drop any NA values
    df.dropna()


In [18]:
#drop nan
AMZN = AMZN.dropna()
META = META.dropna()
TSLA = TSLA.dropna()

# Review complete dfs (with signal)
display(AMZN.head(3))
display(META.head(3))
display(TSLA.head(3))

,open,high,low,close,VWAP,Volume,Volume MA,200EMA,VIX_close,VIX_40_EMA,AMZN_search_trends,pct_returns,Fast_EMA,Slow_EMA,Signal
time,,,,,,,,,,,,,,,
2012-05-02 09:30:00-04:00,11.391001,11.572000,11.370000,11.512500,11.484833,91879460.0,110478210.0,9.684092,16.88,17.494492,40.0,0.000913,11.507833,11.507381,1.0
2012-05-03 09:30:00-04:00,11.487001,11.626500,11.401500,11.472500,11.500167,81157180.0,109076585.0,9.701887,17.56,17.497688,40.0,-0.003474,11.493352,11.495168,-1.0
2012-05-04 09:30:00-04:00,11.390000,11.485995,11.187001,11.199501,11.290832,91757900.0,110446703.0,9.716789,19.16,17.578776,40.0,-0.023796,11.393809,11.415618,-1.0


,open,high,low,close,VWAP,Volume,Volume MA,200EMA,VIX_close,VIX_40_EMA,META_search_trends,pct_returns,Fast_EMA,Slow_EMA,Signal
time,,,,,,,,,,,,,,,
2013-03-08 09:30:00-05:00,28.42500,28.47000,27.73000,27.96001,28.053337,44198832.0,46896756.8,25.692691,12.59,14.269040,91.0,-0.021625,28.234672,28.261280,-1.0
2013-03-11 09:30:00-04:00,28.00999,28.64000,27.83000,28.14000,28.203333,35642064.0,46793423.2,25.717042,11.56,14.136891,91.0,0.006437,28.195872,28.218816,-1.0
2013-03-12 09:30:00-04:00,28.10001,28.32001,27.60001,27.83000,27.916673,27569584.0,46303815.2,25.738067,12.27,14.045823,91.0,-0.011016,28.071932,28.114203,-1.0


,open,high,low,close,VWAP,Volume,Volume MA,200EMA,VIX_close,VIX_40_EMA,TSLA_search_trends,pct_returns,Fast_EMA,Slow_EMA,Signal
time,,,,,,,,,,,,,,,
2012-05-02 09:30:00-04:00,2.233331,2.292664,2.225998,2.262664,2.260442,7458772.0,16338969.05,2.038579,16.88,17.494492,8.0,0.004736,2.257924,2.257464,1.0
2012-05-03 09:30:00-04:00,2.260664,2.266664,2.141998,2.163999,2.190887,12631617.0,13608434.55,2.039827,17.56,17.497688,8.0,-0.043606,2.219430,2.224739,-1.0
2012-05-04 09:30:00-04:00,2.154665,2.163999,2.093331,2.121998,2.126443,18712308.0,13411940.35,2.040645,19.16,17.578776,8.0,-0.019409,2.186425,2.197096,-1.0


## RANDOM FOREST

### Defining and Fitting the Random Forest Model (See Mod14 - Day 3 - Activity 7)

In [19]:
# Define features set (all data columns except for signal)
X = AMZN.drop(columns='Signal')
X.head()

,open,high,low,close,VWAP,Volume,Volume MA,200EMA,VIX_close,VIX_40_EMA,AMZN_search_trends,pct_returns,Fast_EMA,Slow_EMA
time,,,,,,,,,,,,,,
2012-05-02 09:30:00-04:00,11.391001,11.572000,11.370000,11.512500,11.484833,91879460.0,110478210.0,9.684092,16.88,17.494492,40.0,0.000913,11.507833,11.507381
2012-05-03 09:30:00-04:00,11.487001,11.626500,11.401500,11.472500,11.500167,81157180.0,109076585.0,9.701887,17.56,17.497688,40.0,-0.003474,11.493352,11.495168
2012-05-04 09:30:00-04:00,11.390000,11.485995,11.187001,11.199501,11.290832,91757900.0,110446703.0,9.716789,19.16,17.578776,40.0,-0.023796,11.393809,11.415618
2012-05-07 09:30:00-04:00,11.118000,11.343375,11.114500,11.258000,11.238625,66284520.0,110624051.0,9.732124,18.94,17.645177,40.0,0.005223,11.353409,11.380865
2012-05-08 09:30:00-04:00,11.157500,11.269500,10.910000,11.195000,11.124833,90460860.0,110691602.0,9.746680,19.05,17.713705,40.0,-0.005596,11.310471,11.345890


In [20]:
# Define target vector (Signal column only, generated by trading algo above)
y = AMZN['Signal'].astype('int8')
y.head()

time
2012-05-02 09:30:00-04:00    1
2012-05-03 09:30:00-04:00   -1
2012-05-04 09:30:00-04:00   -1
2012-05-07 09:30:00-04:00   -1
2012-05-08 09:30:00-04:00   -1
Name: Signal, dtype: int8

### Create Training Datasets

In [21]:
from pandas.tseries.offsets import DateOffset

# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

# Select the ending period for the training data with an offset of 2 years
training_end = X.index.min() + DateOffset(years=2)

# Display the training end date
print(training_end)

2012-05-02 09:30:00-04:00
2014-05-02 09:30:00-04:00


In [22]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Display sample data
display(X_train.head(3))
display(y_train.head(3))

,open,high,low,close,VWAP,Volume,Volume MA,200EMA,VIX_close,VIX_40_EMA,AMZN_search_trends,pct_returns,Fast_EMA,Slow_EMA
time,,,,,,,,,,,,,,
2012-05-02 09:30:00-04:00,11.391001,11.572000,11.370000,11.512500,11.484833,91879460.0,110478210.0,9.684092,16.88,17.494492,40.0,0.000913,11.507833,11.507381
2012-05-03 09:30:00-04:00,11.487001,11.626500,11.401500,11.472500,11.500167,81157180.0,109076585.0,9.701887,17.56,17.497688,40.0,-0.003474,11.493352,11.495168
2012-05-04 09:30:00-04:00,11.390000,11.485995,11.187001,11.199501,11.290832,91757900.0,110446703.0,9.716789,19.16,17.578776,40.0,-0.023796,11.393809,11.415618


time
2012-05-02 09:30:00-04:00    1
2012-05-03 09:30:00-04:00   -1
2012-05-04 09:30:00-04:00   -1
Name: Signal, dtype: int8

### Create Testing Datasets

In [23]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

# Display sample data
display(X_test.head(3))
display(y_test.head(3))

,open,high,low,close,VWAP,Volume,Volume MA,200EMA,VIX_close,VIX_40_EMA,AMZN_search_trends,pct_returns,Fast_EMA,Slow_EMA
time,,,,,,,,,,,,,,
2014-05-02 09:30:00-04:00,15.5210,15.6645,15.2155,15.4005,15.426833,79904360.0,132165466.0,16.846889,12.91,14.234227,50.0,0.000390,15.534666,16.590825
2014-05-05 09:30:00-04:00,15.3185,15.5115,15.2500,15.5025,15.421333,50406000.0,122151188.0,16.833512,13.29,14.188167,50.0,0.006623,15.528233,16.537736
2014-05-06 09:30:00-04:00,15.4765,15.4905,14.8520,14.8690,15.070500,93646540.0,119756155.0,16.813965,13.80,14.169232,50.0,-0.040864,15.396386,16.456334


time
2014-05-02 09:30:00-04:00   -1
2014-05-05 09:30:00-04:00   -1
2014-05-06 09:30:00-04:00   -1
Name: Signal, dtype: int8

### Standardizing the Data

In [24]:
# Creating StandardScaler instance
scaler = StandardScaler()

In [25]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

In [26]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [27]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [28]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

### Making Predictions Using the Random Forest Model

In [29]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

### Model Evaluation

In [30]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [31]:
# Displaying results
print("Confusion Matrix")
#display(AMZN)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix
Accuracy Score : 0.6742014742014742
Classification Report
              precision    recall  f1-score   support

          -1       0.58      0.12      0.19       685
           1       0.68      0.96      0.80      1350

    accuracy                           0.67      2035
   macro avg       0.63      0.54      0.50      2035
weighted avg       0.65      0.67      0.59      2035



### Feature Importance

In [32]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf_model.feature_importances_

In [33]:

# We can sort the features by their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.18658560956339373, '200EMA'),
 (0.1859867556371003, 'Volume MA'),
 (0.08486531422694422, 'Fast_EMA'),
 (0.07377606212067153, 'Slow_EMA'),
 (0.07284268955876638, 'VIX_40_EMA'),
 (0.06963374311797446, 'open'),
 (0.060169777117187066, 'high'),
 (0.058292155760864064, 'VWAP'),
 (0.05299330982877774, 'low'),
 (0.05281532744994819, 'close'),
 (0.033137564629769904, 'AMZN_search_trends'),
 (0.031289691960863564, 'VIX_close'),
 (0.026380581280623067, 'Volume'),
 (0.01123141774711563, 'pct_returns')]

### Comparative Backtest Results

In [34]:
# Create a new empty predictions DataFrame using code provided below.
pred_amzn_df = pd.DataFrame(index=X_test.index)
pred_amzn_df["rf_signal"] = predictions
pred_amzn_df["basic_signal"] = AMZN['Signal']
pred_amzn_df["close"] = AMZN['close']

#-----------------------------------
# Set the initial capital
initial_capital = float(100000)

# Set position size to 500 shares 
share_size = 200
#-----------------------------------------------------------------------------

### non-ML returns
# Take a share_size position 
pred_amzn_df["basic_position"] = share_size * pred_amzn_df["basic_signal"]
pred_amzn_df["rf_position"] = share_size * pred_amzn_df["rf_signal"]

# Find the points in time where a trade occurs
pred_amzn_df["basic_Entry_Exit_Position"] = pred_amzn_df["basic_position"].diff()
pred_amzn_df["rf_Entry_Exit_Position"] = pred_amzn_df["rf_position"].diff()


# Multiply share price by entry/exit positions and get the cumulatively sum
pred_amzn_df["basic_port_holdings"] = (
    pred_amzn_df["close"] * pred_amzn_df["basic_Entry_Exit_Position"].cumsum()
)
pred_amzn_df["rf_port_holdings"] = (
    pred_amzn_df["close"] * pred_amzn_df["rf_Entry_Exit_Position"].cumsum()
)

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
pred_amzn_df["basic_port_cash"] = (
    initial_capital - (pred_amzn_df["close"] * pred_amzn_df["basic_Entry_Exit_Position"]).cumsum()
)
pred_amzn_df["rf_port_cash"] = (
    initial_capital - (pred_amzn_df["close"] * pred_amzn_df["rf_Entry_Exit_Position"]).cumsum()
)

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
pred_amzn_df["basic_port_total"] = (
    pred_amzn_df["basic_port_cash"] + pred_amzn_df["basic_port_holdings"]
)
pred_amzn_df["rf_port_total"] = (
    pred_amzn_df["rf_port_cash"] + pred_amzn_df["rf_port_holdings"]
)

# Calculate the portfolio daily returns
pred_amzn_df["basic_port_daily_returns"] = pred_amzn_df["basic_port_total"].pct_change()
pred_amzn_df["rf_port_daily_returns"] = pred_amzn_df["rf_port_total"].pct_change()


# Calculate the cumulative returns
pred_amzn_df["basic_port_cumulative_returns"] = (
    1 + pred_amzn_df["basic_port_daily_returns"]
).cumprod() - 1
pred_amzn_df["rf_port_cumulative_returns"] = (
    1 + pred_amzn_df["rf_port_daily_returns"]
).cumprod() - 1


pred_amzn_df.tail(3)
### In this step, add portfolio stats based on non-ML signal (like 02) and ML signal??? 

,rf_signal,basic_signal,close,basic_position,rf_position,basic_Entry_Exit_Position,rf_Entry_Exit_Position,basic_port_holdings,rf_port_holdings,basic_port_cash,rf_port_cash,basic_port_total,rf_port_total,basic_port_daily_returns,rf_port_daily_returns,basic_port_cumulative_returns,rf_port_cumulative_returns
time,,,,,,,,,,,,,,,,,
2022-01-06 09:30:00-05:00,1,-1.0,163.2540,-200.0,200,0.0,0.0,0.0,65301.6,123129.8,88309.9,123129.8,153611.5,0.000000,-0.002864,0.231298,0.536115
2022-01-07 09:30:00-05:00,1,-1.0,162.5540,-200.0,200,0.0,0.0,0.0,65021.6,123129.8,88309.9,123129.8,153331.5,0.000000,-0.001823,0.231298,0.533315
2022-01-10 09:30:00-05:00,1,-1.0,161.4860,-200.0,200,0.0,0.0,0.0,64594.4,123129.8,88309.9,123129.8,152904.3,0.000000,-0.002786,0.231298,0.529043
2022-01-11 09:30:00-05:00,1,-1.0,165.3620,-200.0,200,0.0,0.0,0.0,66144.8,123129.8,88309.9,123129.8,154454.7,0.000000,0.010140,0.231298,0.544547
2022-01-12 09:30:00-05:00,1,-1.0,165.2070,-200.0,200,0.0,0.0,0.0,66082.8,123129.8,88309.9,123129.8,154392.7,0.000000,-0.000401,0.231298,0.543927
2022-01-13 09:30:00-05:00,1,-1.0,161.2140,-200.0,200,0.0,0.0,0.0,64485.6,123129.8,88309.9,123129.8,152795.5,0.000000,-0.010345,0.231298,0.527955
2022-01-14 09:30:00-05:00,1,-1.0,162.1380,-200.0,200,0.0,0.0,0.0,64855.2,123129.8,88309.9,123129.8,153165.1,0.000000,0.002419,0.231298,0.531651
2022-01-18 09:30:00-05:00,1,-1.0,158.9175,-200.0,200,0.0,0.0,0.0,63567.0,123129.8,88309.9,123129.8,151876.9,0.000000,-0.008411,0.231298,0.518769
2022-01-19 09:30:00-05:00,1,-1.0,156.2990,-200.0,200,0.0,0.0,0.0,62519.6,123129.8,88309.9,123129.8,150829.5,0.000000,-0.006896,0.231298,0.508295


In [35]:
# Calculate and plot the cumulative returns for the `actual_returns` and the `trading_algorithm_returns`
pred_amzn_df[["basic_port_total", "rf_port_total"]].hvplot(height=500, width=1200)

:NdOverlay   [Variable]
   :Curve   [time]   (value)